In [1]:
using CSV, DataFrames, JuMP, Gurobi, Formatting, NPZ

# Loading Data

In [2]:
length_s2d_df = CSV.read("csv/l2_s2d.csv", DataFrame, header=false)
length_d2d_df = CSV.read("csv/l2_d2d.csv", DataFrame, header=false);

In [3]:
first(length_d2d_df, 2)

Row,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,4.8211,3.04137,2.33759,1.2254,1.96559,2.38026,2.2427,2.01504
2,4.8211,0.0,1.79856,3.02076,3.81632,2.87258,2.45442,2.59646,2.82419


In [5]:
n_src = nrow(length_s2d_df);
n_dst = ncol(length_s2d_df);

all_src = 1:n_src;
all_dst = 1:n_dst;

println("all_src: $all_src, all_dst: $all_dst")

all_src: 1:3, all_dst: 1:9


In [6]:
L_s2d = Matrix(length_s2d_df);
L_d2d = Matrix(length_d2d_df);

println("L_s2d: $(size(L_s2d)), L_d2d: $(size(L_d2d))")

L_s2d: (3, 9), L_d2d: (9, 9)


# Problem
- We now additionally consider that each robot has a fixed starting source.
- The number of robots is less than the number of destinations.
- Instead of minimizing the total time / distance, we want to minimize the maximum time / distance for any single robot.
- Instead of using the path lengths from pathfinding, we use the Euclidean distance as a proxy.

In [7]:
S = [2, 1, 1];
@assert size(S) == (n_src,)
@assert sum(S) < n_dst

n_robots = sum(S)
all_robots = 1:n_robots;

println("There are a $n_dst destinations but only $n_robots robots to visit them all.")

There are a 9 destinations but only 4 robots to visit them all.


In [8]:
# We assume that each robot visits a maximum of n_steps destinations.
n_steps = 3;
all_steps = 1:n_steps;
later_steps = 1:(n_steps-1);

println("n_steps: $n_steps, all_steps: $all_steps, later_steps: $later_steps")

n_steps: 3, all_steps: 1:3, later_steps: 1:2


# Minimum Euclidean Distance

In [9]:
model2 = Model(Gurobi.Optimizer)

@variable(model2, x0[all_robots, all_src, all_dst] >= 0, Bin);
@variable(model2, x1[all_robots, later_steps, all_dst, all_dst] >= 0, Bin);
@variable(model2, maxdist >= 0);

dist_fromsrc = [sum(sum(L_s2d[ii, jj] * x0[rr, ii, jj] for ii in all_src) for jj in all_dst) for rr in all_robots];
dist_fromdst = [sum(sum(sum(L_d2d[ii, jj] * x1[rr, kk, ii, jj] for ii in all_dst) for jj in all_dst) for kk in later_steps) for rr in all_robots];
dist_total = [dist_fromsrc[rr] + dist_fromdst[rr] for rr in all_robots];

@objective(model2, Min, maxdist);

# The max dist is larger than the dists for each robot.
@constraint(model2, maxdistlb[rr in all_robots], maxdist >= dist_total[rr]);

# Each destination must be served by at least one source.
@constraint(model2, demand[jj in all_dst], sum(sum(x0[rr, ii, jj] for ii in all_src) for rr in all_robots) + sum(sum(sum(x1[rr, kk, ii, jj] for ii in all_dst) for kk in later_steps) for rr in all_robots) >= 1);

# Continuity constraints. x0_{r, ij} = x1_{r, jl}
@constraint(model2, cont0[rr in all_robots, jj in all_dst], sum(x0[rr, ii, jj] for ii in all_src) == sum(x1[rr, 1, jj, ll] for ll in all_dst));
# Continuity constraints. x1_{r, ij} = x2_{r, jl}
@constraint(model2, cont1[rr in all_robots, kk in 1:(n_steps-2), jj in all_dst], sum(x1[rr, kk, ii, jj] for ii in all_dst) == sum(x1[rr, kk + 1, jj, ll] for ll in all_dst));

# Each robot must start from its assigned source.
@constraint(model2, start[ii in all_src], sum(sum(x0[rr, ii, jj] for jj in all_dst) for rr in all_robots) == S[ii]);

# Solve.
optimize!(model2);
solution_summary(model2)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-19
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: AMD Ryzen 9 3950X 16-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 88 rows, 757 columns and 2632 nonzeros
Model fingerprint: 0x9d428d36
Variable types: 1 continuous, 756 integer (756 binary)
Coefficient statistics:
  Matrix range     [2e-01, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 6.6840477
Presolve time: 0.00s
Presolved: 88 rows, 757 columns, 2632 nonzeros
Variable types: 1 continuous, 756 integer (756 binary)

Root relaxation: objective 1.022456e+00, 200 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

* Solver : Gurobi

* Status
  Result count       : 10
  Termination status : OPTIMAL
  Message from the solver:
  "Model was solved to optimality (subject to tolerances), and an optimal solution is available."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 2.35771e+00
  Objective bound    : 2.35771e+00
  Relative gap       : 0.00000e+00
  Dual objective value : 2.35771e+00

* Work counters
  Solve time (sec)   : 1.86416e-01
  Barrier iterations : 0
  Node count         : 98


In [10]:
# Save solution.
x2_0 = Array(value.(x0));
x2_1 = Array(value.(x1));

npzwrite("sols/problem4_l2dist.npz", Dict("x0" => x2_0, "x1" => x2_1, "objective" => objective_value(model2)))